<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TSA_Gold_ds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group - 8
CH24M548 - Mounika Chowdary Pamulapati

CH24M571 - Soumya Mukherjee

CH24M577 - Surya Kandala

Dataset : Gold.csv


Steps followed :
1. **Importing the dataset** :
2. **Cleaning the dataset**: Ensured that missing dates were added and forward fill of the data for the missing dates. Dropped NaN records whereever observed.
3. **Stationartiy Check**: Validated if the timeseries is stationary or not by performing ADF and KSS test. For ADF test if p<0.05, then it says the timeseries is stationary. If p>0.05 for KSS test, it indicates the timeseries is stationary.

4. **First Order Differencing & Log Differencing:** Performing first order differencing to make the timeseries stationary.
5. **ACF and PACF:** Compute Auto Correlation and Partial correlation to understand the presence of MA and AR components.
6. **Power Spectral Density:** For a process to be stationary, both mean and variance need to be independent of the absolute time. Hence, validating if Variance is dependent on the absolute time.
7. **Random Walk:** Based on the ACF and PACF values, going with ARMA model of 010 since ACF and PACF results indicate complete white noise.
8. **Akaike's Information Criteria:** Since the forecast is only a straight line, checked the AIC values for models starting from 000 till 332. Chose that model which has less AIC value
9. **Train, Test & Generate Forecast:** Generated the forecast for 122 model which got less AIC value in step 7.
10. **MSE, RMSE, MAPE:** Identify the Mean Square Error and Root Mean Square Error to check the residual details.
11. **Alternate Non-Lionear models:** Since we have variance as a function of absolute time, linear models might not be the right choice to move ahead, hence trying out the non-linear models ARCH and GARCH to model the data.


--- Starting off by installing dependecies ---


In [ ]:
!pip install pandas seaborn matplotlib

Import dependencies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Use winget to get the data

In [3]:
# Downloading the dataset
!wget 'https://drive.google.com/uc?export=download&id=1cXGNQ3NtUERYHbP5Jm4fmZR1jmd2M9ME' -O Gold.csv

--2024-12-26 20:11:57--  https://drive.google.com/uc?export=download&id=1cXGNQ3NtUERYHbP5Jm4fmZR1jmd2M9ME
Resolving drive.google.com (drive.google.com)... 142.251.163.139, 142.251.163.100, 142.251.163.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1cXGNQ3NtUERYHbP5Jm4fmZR1jmd2M9ME&export=download [following]
--2024-12-26 20:11:57--  https://drive.usercontent.google.com/download?id=1cXGNQ3NtUERYHbP5Jm4fmZR1jmd2M9ME&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.253.115.132, 2607:f8b0:4004:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.253.115.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95758 (94K) [application/octet-stream]
Saving to: ‘Gold.csv’

Gold.csv            100%[===================>]  93.51K  --.-KB/s   

In [4]:
# Load the dataset
try:
    gold_data = pd.read_csv('Gold.csv')
except FileNotFoundError:
    print("Error: 'Gold.csv' not found. Please make sure the file exists and the path is correct.")
    exit()

Dividing the data into test train split using scikit learn . The train_test_split() method is used to split our data into train and test sets.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

gold_data = gold_data.set_index('DATE')  # setting DATE as the index column

# Divide the data into training and testing sets (80:20 split)
train_data, test_data = train_test_split(
        gold_data,
        test_size=0.2,
        shuffle=False
        ) # shuffle=False to maintain time order

# Write the training and testing sets to new CSV files, overwriting if they exist
train_data.to_csv('training.csv', index=False) #index=False to avoid writing row indices
test_data.to_csv('test.csv', index=False) #index=False to avoid writing row indices